In [17]:
import utils.eigenstate_generator as eig2
import utils.cs_vqe as c
import utils.cs_vqe_ansatz as c_anz
import utils.qubit_conversion as qonvert
import ast
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from copy import deepcopy
from openfermion.linalg import LinearQubitOperator, get_sparse_operator, get_ground_state
import itertools
from statistics import median

from qiskit.circuit.parameter import Parameter
from qiskit.circuit.quantumcircuit import QuantumCircuit
from qiskit import QuantumCircuit, BasicAer, execute
from qiskit.visualization import plot_histogram
from qiskit.utils import QuantumInstance
from qiskit.aqua.components.optimizers import SLSQP
from qiskit.algorithms import VQE
from qiskit import Aer
from openfermion.linalg import LinearQubitOperator
from openfermion.ops import FermionOperator, QubitOperator

import refactor.cs_vqe_obj as c_obj
import refactor.eigenstate as eig

In [2]:
import ast
import matplotlib
import matplotlib.pyplot as plt

f = open("hamiltonians/hamiltonians.txt","r")
hamiltonians = ast.literal_eval(f.read())
f.close()

In [3]:
hamiltonians.keys()

dict_keys(['H2-S1_STO-3G_singlet', 'C1-O1_STO-3G_singlet', 'H1-Cl1_STO-3G_singlet', 'H1-Na1_STO-3G_singlet', 'H2-Mg1_STO-3G_singlet', 'H1-F1_3-21G_singlet', 'H1-Li1_3-21G_singlet', 'Be1_STO-3G_singlet', 'H1-F1_STO-3G_singlet', 'H1-Li1_STO-3G_singlet', 'Ar1_STO-3G_singlet', 'F2_STO-3G_singlet', 'H1-O1_STO-3G_singlet', 'H2-Be1_STO-3G_singlet', 'H2-O1_STO-3G_singlet', 'H2_3-21G_singlet', 'H2_6-31G_singlet', 'H3-N1_STO-3G_singlet', 'H4-C1_STO-3G_singlet', 'Mg1_STO-3G_singlet', 'N2_STO-3G_singlet', 'Ne1_STO-3G_singlet', 'O2_STO-3G_singlet', 'H1-Li1-O1_STO-3G_singlet', 'H1-He1_STO-3G_singlet', 'H3_STO-3G_singlet_1+', 'H1-He1_3-21G_singlet_1+', 'H3_3-21G_singlet_1+', 'H4-N1_STO-3G_singlet_1+'])

In [4]:
speciesname = 'H2-O1_STO-3G_singlet'

mol = hamiltonians[speciesname]
ham = mol[2]
num_qubits = mol[1]
#print(speciesname, num_qubits)
terms_noncon = c.greedy_dfs(ham, 1, criterion='weight')[-1]
cs_vqe_mol = c_obj.cs_vqe(ham, terms_noncon, num_qubits)
A = cs_vqe_mol.generators(rot=False)[1]
print(speciesname, num_qubits, A)

H2-O1_STO-3G_singlet 10 {'IIIIZXZZZZ': 9.576858245891391e-08, 'IIIIIZIIII': -0.9999999999999954}


In [5]:
cs_vqe_mol.ep_dist()

{(1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1): 1.1379787092233828e-15,
 (1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1): 0.5000000478842901,
 (1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1): 1.137978491258188e-15,
 (1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1): 0.49999995211570764}

In [34]:
psi = eig.eigenstate(A, 52, num_qubits).construct()

In [35]:
A_q = qonvert.dict_to_QubitOperator(A)

In [36]:
eig2.expectation(A_q, psi, num_qubits)

(0.9999999999999954+0j)